In [2]:
import pandas as pd
import os
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
os.chdir("/Users/kausshik/HON322M/Misc_LLM")

In [4]:
data = pd.read_csv('Hotel_review.csv', index_col=0)

In [5]:
data.head()

,hotel,urls,reviewer,review_overall,title,text,ReviewerValue,ReviewerLocation,ReviewerService,ReviewerRooms,ReviewerCleanliness,ReviewerSleepQuality,Date of Stay,Trip Type,Contributions,Helpful Vote
0,"LondonHouse Chicago, Curio Collection by Hilton",https://www.tripadvisor.com/Hotel_Review-g3580...,Jen H,5.0,Manny the Man,The London House Cupola is breathtaking and th...,NaN,NaN,NaN,NaN,NaN,NaN,December 2021,NaN,4,1.0
1,"LondonHouse Chicago, Curio Collection by Hilton",https://www.tripadvisor.com/Hotel_Review-g3580...,CorMom,5.0,Amazing Views and Location,We stayed here for a family vacation to go to ...,5.0,5.0,5.0,NaN,NaN,NaN,August 2021,NaN,98,79.0
2,"LondonHouse Chicago, Curio Collection by Hilton",https://www.tripadvisor.com/Hotel_Review-g3580...,K W,5.0,Phenomenal Service,I had a wonderful time at the Londonhouse. So ...,5.0,NaN,5.0,5.0,NaN,NaN,December 2021,NaN,1,NaN
3,"LondonHouse Chicago, Curio Collection by Hilton",https://www.tripadvisor.com/Hotel_Review-g3580...,Daniel L,5.0,Great hotel in a great location!,We chose this hotel because it offered great v...,5.0,5.0,5.0,5.0,5.0,5.0,December 2021,NaN,70,6.0
4,"LondonHouse Chicago, Curio Collection by Hilton",https://www.tripadvisor.com/Hotel_Review-g3580...,Himan,5.0,Great customer service,"I had a great time at LondonHouse. The staff, ...",5.0,NaN,5.0,NaN,NaN,5.0,January 2022,NaN,1,NaN


## Pre-processing
1. ***Tokenization*** - The process of breaking down the text into individual words or "tokens".

2. ***Stopwords Removal*** - Eliminating frequently occurring but less meaningful words (e.g., "the," "and," "is") from text data

3. ***Stemming and Lemmatization*** - reduce words to their base or root form. 

    - ***Stemming*** - Simplify words by removing suffixes (e.g., running, runs, and run), but it might result in non-words (e.g., "leaves" -> "leav"). 
    
    - ***Lemmatization*** - on the other hand, transforms words into their original form while retaining their meaning (e.g., "leaves" -> "leaf").

5. ***Vectorization*** - The conversion of text into a numerical format, such as vectors. A common approach is the bag-of-words model, where a matrix is created to store word frequencies (word counts) for each document or text in the corpus. This process is often referred to as the vectorization of the raw text.


### Tokenization


In [6]:
import nltk 

In [8]:
nltk.download('punkt')

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>


False

In [ ]:
# if only apply split to segregate words
first_text = data['text'][0]

print(first_text)
print("="*90)
print(first_text.split(" "))

"comma and period" are not seperated as one term; e.g., "proposal."

nltk library "word_tokenize()": split singular words as well as puctuations into separate elements

In [ ]:
# using nltk.word_tokenize()
first_text_list = nltk.word_tokenize(first_text)

print(first_text_list)
# now it's well seprated.

Another popular pre-processing library for text data is `spacy`.

### Stopword Removal
Stop words include terms such as "to" or "the" and therefore, it would be to our benefit to remove them during the pre-processing phase.

NLTK comes with a list of 179 english stopwords.

In [ ]:
nltk.download('stopwords')

In [ ]:
# using nltk.corpus.stopwords.words('english') to remove stop words
stopwords = nltk.corpus.stopwords.words('english')
len(stopwords)

In [ ]:
print(stopwords)

In [ ]:
# only keep words that do not contain stopwords
first_text_list_cleaned = [word for word in first_text_list if word.lower() not in stopwords]

print(first_text_list_cleaned)
print("="*90)
print("Length of original list: {0} words\n"
     "Length of list after stopwords removal: {1} words"
     .format(len(first_text_list), len(first_text_list_cleaned)))

In [ ]:
# same as [word for word in first_text_list if word.lower() not in stopwords]:
first_text_list_cleaned = []

for word in first_text_list:
    if word.lower() not in stopwords:
        first_text_list_cleaned.append(word)
        
print(first_text_list_cleaned)

### Stemming and Lemmatization
After removal of stopwords, the next stage of NLP that I would like to introduce is the process of Stemming. The work at this stage attempts to reduce as many different variations of similar words into a single term ( different branches all reduced to single word stem). Therefore if we have "running", "runs" and "run", we would really want these three distinct words to collapse into just the word "run". (However of course we lose the past, present or future tense).

In [ ]:
stemmer = nltk.stem.PorterStemmer()

In [ ]:
print("The stemmed form of running is: {}".format(stemmer.stem("running")))
print("The stemmed form of runs is : {}".format(stemmer.stem("runs")))
print("The stemmed form of run is : {}".format(stemmer.stem("run")))

In [ ]:
print("The stemmed form of leaves is: {}".format(stemmer.stem("leaves")))

In [ ]:
nltk.download('wordnet')

In [ ]:
from nltk.stem import WordNetLemmatizer

# define lemm, and use lemm.lemmatize() to do lemmatization
lemm = WordNetLemmatizer()
print("The lemmatized form of leaves is: {}".format(lemm.lemmatize("leaves")))

The lemmatizer is working; making the words much more lexical sense.

### Vectorization

A machine can read in bits and numbers and therefore we will first need to convert our text into numbers (Machine learning algorithms operate on a numeric feature space) for which we utilise a very common approach known as the Bag-of-Words.

***The Bag of Words approach*** -
This approach uses the counts of words and records the occurrence of each word. For example given these two sentences "I love to eat Burgers", "I love to eat Fries", we first tokenize to obtain our vocabulary of 6 words from which we can get the word counts for - [I, love, to, eat, Burgers, Fries].

Each word is a feature and each row is a sentence.

In [ ]:
# CountVectorizer(): Convert a collection of text documents to a matrix of token counts
# Defining our sentence
sentence = ["I love to eat Burgers",
           "I love to eat Fries"]

vectorizer = CountVectorizer(stop_words= "english") 

# fit
sentence_transform = vectorizer.fit_transform(sentence)

In [ ]:
print("The features are: \n {}".format(vectorizer.get_feature_names_out())) # \n: add new line
print("The vectorized array looks like: \n {}".format(sentence_transform.toarray()))

First row: 1 burger, 1 eat, 0 fries, 1 love

Second row: 0 burger, 1 eat, 1 fries, 1 love

### Putting all the preprocessing steps together
Do not need to go through all the steps in tokenization, stopword removals, stemming/lemmatizing, and vectorization.

Sklearn's tokenizer discards all single character terms like ('a', 'w' etc) and also lower cases all terms by default. Filtering out stopwords in Sklearn is as convenient as passing the value 'english' into the argument "stop_words" where a built-in English stopword list is automatically used.

Unfortunately, there is no built-in lemmatizer in the vectorizer while we can extend the CountVectorizer class by overwriting the "build_analyzer" method as follows:

In [ ]:
from nltk.stem import WordNetLemmatizer

# define lemm to do lemmatization
lemm = WordNetLemmatizer()

# define a class to extend the CountVectorizer class with a lemmatizer
class LemmaCountVectorizer(CountVectorizer):

    # build_analyzer(): Return a callable to process input data. 
    # The callable handles that handles preprocessing, tokenization, and n-grams generation.
    def build_analyzer(self):
        analyzer = super(LemmaCountVectorizer, self).build_analyzer()   # self=countvectorizer
        return lambda doc: (lemm.lemmatize(w) for w in analyzer(doc))
        
#  super() method lets you access methods in a parent class. You can think of super() as a way to jump up to 
# view the methods in the class from which another class is inherited.

In [ ]:
text = ["I love to eat Burgers", "I love to eat Fries"]

tf_vectorizer = LemmaCountVectorizer(stop_words = "english")

tf = tf_vectorizer.fit_transform(text)

print(text)
print("="*90)

feature_names = tf_vectorizer.get_feature_names_out()
print(feature_names)
print("="*90)

tf_dense = tf.toarray()
print("Vectors:\n", tf_dense)
print("="*90)



In [ ]:
# get entire text in a list
text = list(data['text'])

# calling the overwritten Count vectorizer
tf_vectorizer = LemmaCountVectorizer(stop_words = "english")
tf = tf_vectorizer.fit_transform(text)

In [ ]:
print(text)
print("="*90)

feature_names = tf_vectorizer.get_feature_names_out()
print(feature_names)
print("="*90)

tf_dense = tf.toarray()
print("Vectors:\n", tf_dense)
print("="*90)

